In [ ]:

import torch
print(torch.cuda.is_available())
print(torch.cuda.get_device_capability())

True
(6, 0)


In [ ]:
%tensorflow_version 2.x

TensorFlow 2.x selected.


In [ ]:
import tensorflow as tf
print(tf.__version__)

2.1.0-rc1


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
!cp "/content/drive/My Drive/koulu_honmia/kandi19/IDA.zip" /content/

In [ ]:
!cp "/content/drive/My Drive/koulu_honmia/kandi19/benthic/loadbm.py" /content/

In [ ]:
%%capture
!unzip IDA.zip

In [ ]:
import numpy as np
import os
import ntpath
import platform
import matplotlib.pyplot as plt
from loadbm import create_df, create_tf_dataset, prepare_for_training


datapath = 'IDA/Separate lists with numbering/Machine learning splits'
img_path = 'IDA/Images/'

split = 1

train_fname = 'train'+str(split)+'.txt'
test_fname = 'test'+str(split)+'.txt'
val_fname = 'val'+str(split)+'.txt'

part_dat = False

df_train = create_df(os.path.join(datapath, train_fname),
                     img_path,
                     partial_dataset=part_dat,
                     seed=123)

df_test = create_df(os.path.join(datapath, test_fname),
                     img_path,
                     partial_dataset=part_dat,
                     seed=123)

df_val = create_df(os.path.join(datapath, val_fname),
                     img_path,
                     partial_dataset=part_dat,
                     seed=123)

from sklearn.utils import shuffle

df_train = shuffle(df_train)
df_test = shuffle(df_test)
df_val = shuffle(df_val)

df_train.head()

,path,label
146689,IDA/Images/Kageronia_fus206/1-Kageronia_fus206...,17
180766,IDA/Images/Leuctra_hip36/1-Leuctra_hip36.1.201...,21
136286,IDA/Images/Isoperla142/1-Isoperla142.1.2016-10...,16
92449,IDA/Images/Ephemerella_muc74/1-Ephemerella_muc...,10
311025,IDA/Images/Taeniopteryx_neb112/0-Taeniopteryx_...,39


In [ ]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
IMSIZE = (224,224,3)
BATCH_SIZE = 256

train_ds = create_tf_dataset(df_train, imsize=IMSIZE, onehot=True)

val_ds = create_tf_dataset(df_val, imsize=IMSIZE, onehot=True)


train_ds = prepare_for_training(train_ds, 
                                shuffle_buffer_size=1000,
                                batch_size=BATCH_SIZE)

val_ds = prepare_for_training(val_ds, 
                              shuffle_buffer_size=1000,
                              batch_size=BATCH_SIZE)

for image, label in train_ds.take(5):
    print(image.shape)
    print(label.shape)


(256, 224, 224, 3)
(256, 39)
(256, 224, 224, 3)
(256, 39)
(256, 224, 224, 3)
(256, 39)
(256, 224, 224, 3)
(256, 39)
(256, 224, 224, 3)
(256, 39)


In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D


def get_pretrained(imsize=(224, 224, 3), classes=39):
        base_model = InceptionV3(input_shape = imsize, 
                                 weights='imagenet', 
                                 include_top=False)
        
        base_model.trainable = True
        
        x = base_model.output
        x = GlobalAveragePooling2D()(x)
        x = Dense(256, activation='relu')(x)
        predictions = Dense(classes, activation='softmax')(x)
        
        model = Model(inputs=base_model.input, outputs=predictions)
        
        return model

model = get_pretrained()
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy',
                  metrics=['accuracy'])


In [ ]:
from tensorflow.keras.callbacks import CSVLogger
import datetime

csv_logger = CSVLogger('13-01-2020_colab.log',append=True)

tr_steps = len(df_train)//BATCH_SIZE
val_steps = len(df_val)//BATCH_SIZE

model.fit(train_ds, 
          validation_data= val_ds, 
          steps_per_epoch= tr_steps, 
          epochs = 10,
          validation_steps = val_steps,
          callbacks=[csv_logger])


Train for 1255 steps, validate for 179 steps
Epoch 1/10
1255/1255 [==============================] - 1241s 989ms/step - loss: 0.2307 - accuracy: 0.9281 - val_loss: 0.9436 - val_accuracy: 0.7758
Epoch 2/10
1255/1255 [==============================] - 1229s 979ms/step - loss: 0.0539 - accuracy: 0.9825 - val_loss: 0.8371 - val_accuracy: 0.8275
Epoch 3/10
1255/1255 [==============================] - 1223s 974ms/step - loss: 0.0383 - accuracy: 0.9880 - val_loss: 0.7146 - val_accuracy: 0.8486
Epoch 4/10
1255/1255 [==============================] - 1223s 975ms/step - loss: 0.0323 - accuracy: 0.9899 - val_loss: 0.9838 - val_accuracy: 0.8132
Epoch 5/10
1255/1255 [==============================] - 1225s 976ms/step - loss: 0.0264 - accuracy: 0.9918 - val_loss: 0.7923 - val_accuracy: 0.8335
Epoch 6/10
1255/1255 [==============================] - 1225s 976ms/step - loss: 0.0211 - accuracy: 0.9935 - val_loss: 0.9353 - val_accuracy: 0.8268
Epoch 7/10
1255/1255 [==============================] - 1223s